<a href="https://colab.research.google.com/github/NahuelRepetto/Programacion-Concurrente/blob/main/TrabajoN1Parte2Cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
%%writefile Matrices.cpp
#include <iostream>
#include <cstdlib>
#include <thread>
#include <vector>
#include <random>

using namespace std;

#define MAX_DIMENSION 20
#define MIN_DIMENSION 5
#define MAX_VALOR 32
#define MIN_VALOR -32

void inicializarMatriz(int n, int ***matriz, int min, int max)
{
  (*matriz) = (int**)malloc(n*sizeof(int*));    
  random_device device;
  mt19937 generador(device());
  uniform_int_distribution<> distribucion(min, max);    
  for(int i=0; i<n; i++)
  {
    (*matriz)[i] = (int*)malloc(n*sizeof(int));
    for(int j=0; j<n; j++)          
      (*matriz)[i][j] = distribucion(generador);         
  }  
}

void calcularProducto(int n, int fila, int **matrizA, int **matrizB, int **matrizC)
{
  for(int i=0; i<n; i++)
    for(int j=0; j<n; j++)
      matrizC[fila][i] += matrizA[fila][j] * matrizB[j][i];       
}

void mostrarMatriz(int **matriz, int n, string nombre)
{
  cout<<endl<<"Matriz "<<nombre<<":"<<endl;
  for(int i=0; i<n; i++)
  {    
    cout<<"| ";
    for(int j=0; j<n; j++)
    {
      if(matriz[i][j] < -999)
        cout<<matriz[i][j]<<" ";
      else if (matriz[i][j] < -99 || matriz[i][j] > 999)
        cout<<" "<<matriz[i][j]<<" ";    
      else if (matriz[i][j] < -9 || matriz[i][j] > 99)
        cout<<"  "<<matriz[i][j]<<" "; 
      else if (matriz[i][j] < 0 || matriz[i][j] > 9)
        cout<<"   "<<matriz[i][j]<<" ";
      else if (matriz[i][j] < 10)
        cout<<"    "<<matriz[i][j]<<" ";                                  
    }       
    cout<<"  |"<<endl;
  }
}

void compararMatricesCuadradas(int n, int **matrizA, int **matrizB)
{      
  for(int i=0; i<n; i++)
    for(int j=0; j<n; j++)
    {
      if(matrizA[i][j] != matrizB[i][j]) 
      {
        cout<< endl <<"Las matrices no son iguales, se encontro una diferencia en la posicion [" << i <<"][" << j <<"]";   
        return;
      }         
    } 
  cout<< endl <<"Las matrices son iguales !!!";   
}

void liberarMatriz(int n, int ***matriz)
{       
  for(int i=0; i<n; i++)  
    free((*matriz)[i]);      
  free(*matriz);
}

int main(int argc, char *argv[])
{      
  //VERIFICO QUE SE HAYA ENVIADO EL NUMERO DE FILAS/COLUMNAS
  if( argc < 2 ) 
  {
    cout<<"Debe ingresar el numero de filas/columnas, dentro del rango de " <<MIN_DIMENSION<< " a " << MAX_DIMENSION <<endl;
    exit(1);
  }

  int n = atoi(argv[1]); //DIMENSIN DE LA MATRIZ    

  //VERIFICO QUE SE HAYA ENVIADO EL NUMERO DENTRO DE LOS LIMITES ESTABLECIDOS
  if( n < MIN_DIMENSION || n > MAX_DIMENSION ) 
  {
    cout<<"El numero de filas/columnas debe estar dentro del rango de " <<MIN_DIMENSION<< " a " << MAX_DIMENSION <<endl; 
    exit(1);
  }    

  //CREO LAS Matrices
  int **A, **B, **CS, **CH;   

  //INICIALIZO LAS MATRICES, A Y B CON VALORES RANDOM, CS Y CH EN 0
  inicializarMatriz(n, &A, MIN_VALOR, MAX_VALOR);
  inicializarMatriz(n, &B, MIN_VALOR, MAX_VALOR);
  inicializarMatriz(n, &CS, 0, 0);
  inicializarMatriz(n, &CH, 0, 0);
                
  //CALCULO VALORES DE CS (SECUENCIAL)
  for(int i=0; i<n; i++)
    calcularProducto(n, i, A, B, CS);    
    
  //CALCULO VALORES DE CH (CON CONCURRENCIA)
  //CREO UN VECTOR PARA ALMACENAR LAS REFERENCIAS A LOS HILOS CREADOS
  vector<thread> hilos;

  //CREO N HILOS
  for(int i=0; i<n; i++)     
    hilos.push_back(thread(calcularProducto, n, i, A, B, CH));                 
           
  //ESPERO A LOS N HILOS 
  for(int i=0; i<n; i++) 
    hilos[i].join();

  //MUESTRO LOS RESULTADOS
  mostrarMatriz(A,n,"A");    
  mostrarMatriz(B,n,"B");
  mostrarMatriz(CS,n,"CS"); 
  mostrarMatriz(CH,n,"CH"); 

  //COMPARO MATRICES
  compararMatricesCuadradas(n, CS, CH);

  //LIBRERO EL ESPACIO DE LAS MATRICES
  liberarMatriz(n, &A);
  liberarMatriz(n, &B);
  liberarMatriz(n, &CS);
  liberarMatriz(n, &CH);   

  return 0;
}

Overwriting Matrices.cpp


In [18]:
!g++ -pthread -o Matrices Matrices.cpp

In [19]:
!./Matrices 10


Matriz A:
|   -25   -11    17    -8   -15   -31   -19    -8   -31    21   |
|    23    19   -22   -24    20    20   -31    25   -13    20   |
|    14     6    17   -24    28   -15    -7   -16    -8   -11   |
|    11   -22     1   -32    31    16    31    25    30    21   |
|   -13    30   -13    12   -23    15    -4    13    -7    30   |
|   -20    13     1    18    -8    16    17   -19    15    25   |
|    16   -28    -9    16   -17    18    -4    13    -6    30   |
|    29     5   -21    -8     6   -31    30   -22   -17     4   |
|   -22    -8   -17   -28    28    31     7   -11   -17    17   |
|    -6     6    17     3    16     0   -24    11    26    -5   |

Matriz B:
|     3     4   -12    10     6    12    -1   -13   -28    14   |
|    25    18    32    11   -29   -10   -11   -17    30    17   |
|    27   -16   -30    -5   -20    17    27   -22    25    11   |
|    22    28   -30   -23    14   -10   -25    18   -21    21   |
|   -19   -23   -15    31    -8     1   -10    16   -1